<a href="https://colab.research.google.com/github/Marvin2798/roboreviews-project-/blob/main/roboreviewsproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install packages

In [11]:
import os
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, Trainer, TrainingArguments

import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

load datasets

In [12]:
dataset_directory = "/root/.cache/kagglehub/datasets/datafiniti/consumer-reviews-of-amazon-products/versions/5/"
filenames = [
    "Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv",
    "1429_1.csv",
    "Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products_May19.csv"
]

# Loading the datasets
dataframes = []
for file in filenames:
    file_path = os.path.join(dataset_directory, file)
    if os.path.exists(file_path):
        df = pd.read_csv(file_path, encoding='utf-8')
        dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True).drop_duplicates()
print("Combined Dataset Info:")
print(combined_df.info())
combined_df.to_csv("combined_dataset.csv", index=False)


<ipython-input-12-705748d7f6f4>:14: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='utf-8')


Combined Dataset Info:
<class 'pandas.core.frame.DataFrame'>
Index: 67897 entries, 0 to 67991
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    67897 non-null  object 
 1   dateAdded             33237 non-null  object 
 2   dateUpdated           33237 non-null  object 
 3   name                  61137 non-null  object 
 4   asins                 67895 non-null  object 
 5   brand                 67897 non-null  object 
 6   categories            67897 non-null  object 
 7   primaryCategories     33237 non-null  object 
 8   imageURLs             33237 non-null  object 
 9   keys                  67897 non-null  object 
 10  manufacturer          67897 non-null  object 
 11  manufacturerNumber    33237 non-null  object 
 12  reviews.date          67858 non-null  object 
 13  reviews.dateAdded     25083 non-null  object 
 14  reviews.dateSeen      67897 non-null  object 
 15  r

Data Preprocessing

In [13]:
keywords = ['smart speaker', 'Alexa', 'Siri', 'Google Home', 'Bose', 'Sonos', 'Bluetooth Speaker']
smart_speakers = combined_df[combined_df.apply(
    lambda row: row.astype(str).str.contains('|'.join(keywords), case=False, na=False).any(),
    axis=1
)].drop_duplicates()

print("Filtered Smart Speaker Reviews:")
print(smart_speakers.head())
smart_speakers.to_csv("smart_speaker_reviews.csv", index=False)


Filtered Smart Speaker Reviews:
                       id             dateAdded           dateUpdated  \
96   AWFUWc8THh53nbDRF6YO  2018-02-02T02:30:22Z  2018-10-15T16:03:30Z   
97   AWFUWc8THh53nbDRF6YO  2018-02-02T02:30:22Z  2018-10-15T16:03:30Z   
98   AWFUWc8THh53nbDRF6YO  2018-02-02T02:30:22Z  2018-10-15T16:03:30Z   
99   AWFUWc8THh53nbDRF6YO  2018-02-02T02:30:22Z  2018-10-15T16:03:30Z   
100  AWFUWc8THh53nbDRF6YO  2018-02-02T02:30:22Z  2018-10-15T16:03:30Z   

                                                  name       asins   brand  \
96   Amazon Echo Show Alexa-enabled Bluetooth Speak...  B010CEHQTG  Amazon   
97   Amazon Echo Show Alexa-enabled Bluetooth Speak...  B010CEHQTG  Amazon   
98   Amazon Echo Show Alexa-enabled Bluetooth Speak...  B010CEHQTG  Amazon   
99   Amazon Echo Show Alexa-enabled Bluetooth Speak...  B010CEHQTG  Amazon   
100  Amazon Echo Show Alexa-enabled Bluetooth Speak...  B010CEHQTG  Amazon   

                                            categories     p

Cleaning

In [14]:
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])
    return text

smart_speakers['cleaned_text'] = smart_speakers['reviews.text'].apply(clean_text)


Sentiment Labels

In [15]:
def sentiment_label(rating):
    if rating >= 4:
        return "positive"
    elif rating == 3:
        return "neutral"
    else:
        return "negative"

smart_speakers['sentiment'] = smart_speakers['reviews.rating'].apply(sentiment_label)


Split data

In [16]:
train, test = train_test_split(smart_speakers, test_size=0.2, random_state=42)
print("Training Set Size:", train.shape)
print("Testing Set Size:", test.shape)


Training Set Size: (9189, 29)
Testing Set Size: (2298, 29)


Sentiment classification

In [17]:
# Initialize HuggingFace tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

# Tokenize the training and testing data
train_encodings = tokenizer(list(train['cleaned_text']), truncation=True, padding=True, max_length=256)
test_encodings = tokenizer(list(test['cleaned_text']), truncation=True, padding=True, max_length=256)

# Fine-tuning and training logic here (use Trainer from HuggingFace)
#prediction - Test training for performance of model
#fine tune and check performance if bad
#visual inspection ( check manually)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
# Map sentiment labels to integers
label_mapping = {'positive': 0, 'neutral': 1, 'negative': 2}
smart_speakers['label'] = smart_speakers['sentiment'].map(label_mapping)

# Split into training and testing sets
from sklearn.model_selection import train_test_split

train, test = train_test_split(smart_speakers, test_size=0.2, random_state=42)


In [35]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenize the training and testing datasets
train_encodings = tokenizer(list(train['cleaned_text']), truncation=True, padding=True, max_length=256)
test_encodings = tokenizer(list(test['cleaned_text']), truncation=True, padding=True, max_length=256)


In [36]:
import torch

class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# Prepare datasets
train_labels = train['label'].tolist()
test_labels = test['label'].tolist()

train_dataset = SentimentDataset(train_encodings, train_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)


In [37]:
from transformers import AutoModelForSequenceClassification

# Load pre-trained DistilBERT with a classification head
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # Directory to save the model checkpoints
    evaluation_strategy="epoch",    # Evaluate after each epoch
    learning_rate=2e-5,             # Learning rate
    per_device_train_batch_size=8,  # Batch size per device during training
    num_train_epochs=3,             # Number of epochs to train
    weight_decay=0.01,              # Regularization strength
    logging_dir='./logs',           # Directory for logging
    logging_steps=500,              # Log progress every 500 steps
    save_strategy="epoch",          # Save model checkpoint after each epoch
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer

# Initialize the Trainer
trainer = Trainer(
    model=model,                     # The model to fine-tune
    args=training_args,              # Training configuration
    train_dataset=train_dataset,     # Training dataset
    eval_dataset=test_dataset,       # Evaluation dataset
    tokenizer=tokenizer              # Tokenizer
)

# Start training
trainer.train()


<ipython-input-39-d39af03a4f82>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss


In [33]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load pretrained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)  # Adjust `num_labels` based on the task


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
import sklearn
import sklearn.metrics

In [32]:
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',          # Directory to save checkpoints and logs
    evaluation_strategy="epoch",    # Evaluate the model after every epoch
    learning_rate=2e-5,             # Learning rate for the optimizer
    per_device_train_batch_size=8,  # Batch size for training
    per_device_eval_batch_size=8,   # Batch size for evaluation
    num_train_epochs=3,             # Number of epochs
    weight_decay=0.01,              # Regularization strength
    save_steps=10_000,              # Save checkpoints every 10,000 steps
    save_total_limit=2,             # Keep only 2 latest checkpoints
    logging_dir='./logs',           # Log directory
    logging_steps=500,              # Log training progress every 500 steps
)

# Define the Trainer
trainer = Trainer(
    model=model,                     # Pre-trained model (DistilBERT in this case)
    args=training_args,              # Training arguments
    train_dataset=train_dataset,     # Training dataset
    eval_dataset=test_dataset,       # Evaluation dataset
    tokenizer=tokenizer              # Tokenizer used for text processing
)

# Train the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-32-89bf7b7b7242>:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [18]:
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = sbert_model.encode(smart_speakers['cleaned_text'].tolist())

kmeans = KMeans(n_clusters=3, random_state=42)
smart_speakers['cluster'] = kmeans.fit_predict(embeddings)

print(smart_speakers['cluster'].value_counts())


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

cluster
0    4052
1    3729
2    3706
Name: count, dtype: int64


Summarization

In [23]:
from transformers import pipeline

# Load the summarizer model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Loop through each cluster and generate summaries
for cluster in smart_speakers['cluster'].unique():
    # Filter reviews for the current cluster
    cluster_reviews = smart_speakers[smart_speakers['cluster'] == cluster]['cleaned_text'].tolist()

    if len(cluster_reviews) == 0:
        print(f"Cluster {cluster} has no reviews.")
        continue  # Skip empty clusters

    # Combine reviews into a single text (truncate to 500 words for summarization)
    combined_text = " ".join(cluster_reviews)
    if len(combined_text.split()) > 500:  # Limit input to 500 words
        combined_text = " ".join(combined_text.split()[:500])

    # Generate the summary
    try:
        summary = summarizer(combined_text, max_length=100, min_length=30, do_sample=False)
        print(f"Cluster {cluster} Summary:", summary[0]['summary_text'])
    except Exception as e:
        print(f"Error summarizing cluster {cluster}: {e}")



Cluster 2 Summary: Alexa plus hd video always ready answers associated video text applicable show movie trailers also watch amazon video excellent ondemand security video amazon compatible cameras voice activated message andor video calls amazon show owners highly recommended one alexa three echo dots echo show awesome great looking design amazon echo show easy set use overall product excellent speakers capability.
Cluster 0 Summary: The author bought 4 christmas gifts for her family. She also bought a smart home control device for her son. The author also bought twofor deal presents for herself.
Cluster 1 Summary: Amazon responsive program limited information program user friendly love kitchen replaced ipad used use everything amazon even see front door arlo doesnt netflix hulu thats okay really like sound fills small rooms easy video chat nana drop tell kids goodnight working late nice device good enable drop conversations flash briefings overall still little limited effective use vi

In [24]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans

# Generate embeddings
sbert_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = sbert_model.encode(smart_speakers['cleaned_text'].tolist())

# Apply K-Means clustering
num_clusters = 3  # Adjust based on your needs
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
smart_speakers['cluster'] = kmeans.fit_predict(embeddings)


In [25]:
print(smart_speakers['cluster'].value_counts())


cluster
0    4052
1    3729
2    3706
Name: count, dtype: int64
